In [2]:
from pathlib import Path

import polars as pl
import srsly

In [3]:
raw_tok_path = Path("/home/pl487/rdd/outputs/tok_train/bpe_minipile_2024-09-22T17-58-54")

In [10]:
vocab_size = 32_000
tok_type = "bpe"
dataset = "minipile"
tok_path = Path("outputs/tokenizers") / f"{tok_type}{vocab_size}{dataset}"
with (tok_path / "raw_tok_path.txt").open("r") as fl:
    p = fl.readline()

In [11]:
df = pl.DataFrame(srsly.read_jsonl(Path(raw_tok_path) / "implemented_merges.jsonl"))

In [12]:
df = df.with_columns(prefix=pl.col("pair").list.get(0))

In [13]:
df = df.filter(pl.col("new_token_id") < vocab_size)

In [14]:
df = df.group_by("prefix").agg(pl.col("new_token_id")).sort("prefix")

In [15]:
df.write_ndjson(tok_path / f"prefix_map_{tok_type}{vocab_size}{dataset}.jsonl")

In [9]:
(
    df.with_columns(pl.col("new_token_id").list.min())
    .with_columns(check=pl.col("prefix") >= pl.col("new_token_id"))["check"]
    .sum()
)

0